In [2]:
#!pip install gensim
#https://radimrehurek.com/gensim/models/keyedvectors.html
#nltk.download('punkt')

In [3]:
#!pip install html2text
#!python -m spacy link en_core_web_md en_default

In [1]:
import pandas as pd
import os
from bs4 import BeautifulSoup
import time
import spacy
import re
import nltk

df_train = pd.read_csv('share_repurchase_paragraphs.csv')
df_train = df_train.apply(lambda x: x.astype(str).str.lower())

In [62]:
start = time.time()

path = "data/nc_validation_filings_test/"
file_pattern = r"[\d-]+"

acc_df = pd.DataFrame()
dict_zip = {}

for filename in os.listdir(path):
     if filename.endswith('.html'):
        fname = os.path.join(path,filename)
        with open(fname, 'r', encoding="utf8") as f:
            soup = BeautifulSoup(f.read(),'html.parser')
            soup.head.extract()
            soup = soup.get_text().strip()
            sentences = nltk.sent_tokenize(soup)
            key = re.findall(file_pattern, fname)
            values = [sentence for sentence in sentences if "repurchase" in sentence]
            

            
#head_reg = r"<head>.*</head>"
#soup = re.sub(head_reg, '', str(soup), 0, re.MULTILINE)

end = time.time()
print(end-start)
#print(soup)

4.04426121711731


--> next steps - pass data to df or dict for iteration

In [168]:
soup = soup.strip()

'UNITED STATES SECURITIES AND EXCHANGE\nCOMMISSION\nWashington, D.C. 20549\nFORM 10-K\n\ufeff\n☒ ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE\nSECURITIES EXCHANGE ACT OF 1934\nFor the fiscal year ended December 31, 2017\nor\n\ufeff\n☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF\nTHE SECURITIES EXCHANGE ACT OF 1934\nFor the transition period from _________ to ___________\n\ufeff\nCommission file number 001-11001\n\ufeff\n\n\ufeff\nFRONTIER COMMUNICATIONS CORPORATION\n(Exact name of registrant as specified in its charter)\n\ufeff\n\n\n\n\n\ufeff\n\n\n\n\n\n\n\n\n\n\nDelaware\n\n\n\n\n\n06-0619596\n\n\n\n\n(State or other jurisdiction of\n\n\n\n\n\n(I.R.S. Employer Identification No.)\n\n\n\n\nincorporation or organization)\n\n\n\n\n\n\n\n\n\n\n\ufeff\n\n\n\n\n\n\n\n\n\n\n401 Merritt 7\n\n\n\n\n\n\n\n\n\n\nNorwalk, Connecticut\n\n\n\n\n\n06851\n\n\n\n\n(Address of principal executive offices)\n\n\n\n\n\n(Zip Code)\n\n\n\n\n\ufeff\n\n\n\n\n\n\n\n\n\n\nRegistrant\'s telepho

In [154]:
##########################################
#    pasting for reference purposes     #
########################################

# https://spacy.io/usage/training#ner
#!/usr/bin/env python
# coding: utf8
"""Example of training spaCy's named entity recognizer, starting off with an
existing model or a blank model.

For more details, see the documentation:
* Training: https://spacy.io/usage/training
* NER: https://spacy.io/usage/linguistic-features#named-entities

Compatible with: spaCy v2.0.0+
"""

from __future__ import unicode_literals, print_function

import plac
import random
from pathlib import Path
import spacy


# training data
TRAIN_DATA = [
    ('Who is Shaka Khan?', {
        'entities': [(7, 17, 'PERSON')]
    }),
    ('I like London and Berlin.', {
        'entities': [(7, 13, 'LOC'), (18, 24, 'LOC')]
    })
]


@plac.annotations(
    model=("Model name. Defaults to blank 'en' model.", "option", "m", str),
    output_dir=("Optional output directory", "option", "o", Path),
    n_iter=("Number of training iterations", "option", "n", int))
def main(model=None, output_dir=None, n_iter=100):
    """Load the model, set up the pipeline and train the entity recognizer."""
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank('en')  # create blank Language class
        print("Created blank 'en' model")

    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe('ner')

    # add labels
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                    [text],  # batch of texts
                    [annotations],  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
            print(losses)

    # test the trained model
    for text, _ in TRAIN_DATA:
        doc = nlp(text)
        print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
        print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

        # test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        for text, _ in TRAIN_DATA:
            doc = nlp2(text)
            print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
            print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])


#if __name__ == '__main__':
#    plac.call(main)

    # Expected output:
    # Entities [('Shaka Khan', 'PERSON')]
    # Tokens [('Who', '', 2), ('is', '', 2), ('Shaka', 'PERSON', 3),
    # ('Khan', 'PERSON', 1), ('?', '', 2)]
    # Entities [('London', 'LOC'), ('Berlin', 'LOC')]
    # Tokens [('I', '', 2), ('like', '', 2), ('London', 'LOC', 3),
    # ('and', '', 2), ('Berlin', 'LOC', 3), ('.', '', 2)]

In [147]:
#df_train['paragraph_text'][1]

In [146]:
#df_train[df_train['ticker'] == 'aapl']

In [145]:
#df_train.data_key_friendly_name.value_counts()

In [144]:
#df_train[df_train["data_key_friendly_name"] == 'share repurchase authorization']

In [143]:
#df_train[df_train["data_key_friendly_name"] == 'share repurchase authorization'].text.unique()

In [148]:
nlp = spacy.load('en')
doc = nlp(soup)

# Print all of the found entities and their labels
for ent in doc.ents:
    print(ent.label_, ent.text)


ORDINAL 











ORG UNITED STATES SECURITIES AND EXCHANGE
COMMISSION
GPE 

GPE Washington
GPE D.C.
CARDINAL 20549

GPE 

GPE 

ORG ANNUAL
ORG THE
SECURITIES EXCHANGE ACT
DATE the fiscal year ended December 31, 2017

FAC 
﻿
☐ TRANSITION REPORT PURSUANT
DATE 1934
ORG _
ORG _
ORG _
PERSON _
PERSON _
ORDINAL _
ORG 
﻿
Commission
MONEY 001-11001

ORG 
FRONTIER COMMUNICATIONS CORPORATION

ORG Exact
GPE 

GPE Delaware
CARDINAL 06-0619596
GPE 




(State
PERSON 






ORG I.R.S. Employer Identification
GPE 





DATE 










﻿
CARDINAL 401
PRODUCT Merritt 7
GPE Norwalk
GPE Connecticut
GPE 





PERSON 






PERSON Zip Code
GPE 





CARDINAL 203
DATE 614-5600

ORG 
Securities
ORG 










Title
PERSON 





Name
ORG 




Common Stock
MONEY 0.25
GPE 

CARDINAL 






ORG The NASDAQ Stock Market LLC




Series A
PERCENT 11.125%
ORG Mandatory Convertible Preferred Stock
EVENT Series
ORG The NASDAQ Stock Market LLC

ORG The NASDAQ Global Select Market




﻿

LAW Section 12(g
GPE 

GPE 

G

MONEY $75 million
GPE 

CARDINAL 17
MONEY $28 million
DATE 2017
DATE 2016
GPE 

DATE future years
GPE 

GPE 

GPE 

GPE 

GPE 

GPE 

EVENT 
Pension
GPE 

GPE 

GPE 

CARDINAL 18
ORG FRONTIER COMMUNICATIONS CORPORATION
GPE 

GPE 

GPE 

GPE 

GPE 

GPE 

GPE 

ORG Frontier
MONEY $165 million
PERCENT 2%
DATE 2017
GPE 

MONEY $170 million
PERCENT 2%
DATE 2016
GPE 

GPE 

ORG Frontier
GPE 

ORG Frontier
GPE 

ORG FCC
GPE 

GPE 

GPE 

DATE 2011
ORG FCC
DATE 2011
ORG Intercarrier Compensation
GPE 

GPE 

ORG The 2011 Order
CARDINAL zero
GPE 

DATE 2017
DATE 2018
ORG VoIP

GPE 

ORG FCC
GPE 

ORG the Universal Service Fund
GPE 

GPE 

GPE 

ORG the 2011 Order
ORG Intercarrier Compensation
ORG The FCC

GPE 

ORG FCC
GPE 

GPE 

GPE 

GPE 

DATE August 2017
ORG FCC
GPE 

GPE 

GPE 

GPE 

GPE 

GPE 

GPE 

ORG FCC
GPE 

GPE 

GPE 

GPE 

GPE 

GPE 

GPE 

ORG Frontier
GPE 

GPE 

GPE 

GPE 

ORG Frontier
MONEY $395 million
PERCENT 4%
DATE 2017
MONEY $409 million
PERCENT 5%
GPE 

DATE 2016
ORG


ORG 












































ORG 












































PERSON ﻿






DATE the year ended December 31
MONEY 





$ Increase
PERSON 








MONEYGPE 

DATE 2015
CARDINAL 250,000
GPE 

ORG the CTF Acquisition
DATE 2016
GPE 

CARDINAL 38,000
GPE 

DATE 2016
ORG CTF

DATE monthly
ORG ARPC
GPE 

MONEY 17.16
PERCENT 2%
MONEY 673.72
GPE 

DATE 2016
DATE 2015
ORG ARPC
GPE 

ORG CTF
ORG ARPC
ORG SME
GPE 

GPE 

GPE 

GPE 

DATE 2016
PERCENT 8%
GPE 

ORG Frontier
DATE 2016
GPE 

GPE 

DATE December 31, 2016
CARDINAL approximately 1.8 million
GPE 

DATE December 31, 2015
CARDINAL 2,052,000
GPE 

CARDINAL approximately 243,000
GPE 

DATE December 31, 2016
PERCENT 57%
CARDINAL 6
PERCENT 29%
DATE December 31
GPE 

DATE 2015
GPE 

GPE 

PERSON Vantage
GPE 

GPE Connecticut
GPE South Carolina
GPE Minnesota
GPE Illinois
LOC FiOS
GPE 

GPE California
GPE Texas
GPE Florida
GPE Indiana
GPE Oregon
GPE 

GPE Washington
GPE 

DATE the year
GPE 

CARDINAL 1,165,00

ORG Current Report
DATE January
3, 1995
ORG 





CARDINAL 4.3
DATE August 1991
DATE June 15, 1995
GPE 

ORG Frontier
ORG JPMorgan Chase Bank, N.A.
ORG Chemical Bank
ORG Trustee
GPE 

PERCENT 7.45%
DATE 2035
ORG Exhibit 4.100.8
ORG Frontier
ORG Current Report
DATE March 29,
1996
DATE March 29, 1996
DATE August 1991
DATE October 15,
1995
ORG Frontier
ORG JPMorgan Chase Bank, N.A.
ORG Chemical Bank
ORG Trustee
GPE 

PERCENT 7%
DATE 2025
ORG Exhibit 4.100.9
DATE the March 29, 1996 8-K
ORG 





CARDINAL 4.5
ORG 


Seventh Supplemental Indenture
DATE August 1991
DATE June 1,
1996
ORG Frontier
ORG JPMorgan Chase Bank, N.A.
ORG Chemical Bank
ORG Trustee
GPE 

PERCENT 6.8%
DATE 2026
ORG Frontier
ORG Annual Report
DATE Form 10-K
DATE the year
ended December 31, 1996
CARDINAL 56
ORG 


Eighth Supplemental Indenture
DATE August 1991
DATE December 1,
1996
ORG Frontier
ORG JPMorgan Chase Bank, N.A.
ORG Chemical Bank
ORG Trustee
GPE 

PERCENT 7.05%
DATE 2046
ORG Exhibit
DATE 1996
GPE 





CARDINAL


MONEY $ in millions)

DATE ﻿




﻿
ORG Entities
PERSON Consolidated Frontier
ORG 











































Current
MONEY 





$
ORG 


-
CARDINAL 362
ORG 





MONEY 



CARDINAL 362
MONEY $69








CARDINAL 650
QUANTITY 199







CARDINAL 30
GPE 








-
CARDINAL 67
DATE 







-








DATE 








-







109
DATE 







43
DATE 21







-







64
PERSON 








CARDINAL 693
QUANTITY 758







GPE 







CARDINAL 1,323
GPE ﻿











































Property
DATE 







11,546
DATE 2,831







-








CARDINAL 14,377
PERSON Goodwill
DATE 







5,457
NORP 







DATE 







-







7,024
DATE 







1,923
NORP 







DATE 







-








CARDINAL 2,063
GPE 







DATE 







31
DATE 66







-








PERSON 








CARDINAL 3,486
CARDINAL 1,788
GPE 







CARDINAL 11,879
DATE 







-








PERSON 





$
CARDINAL 16,149
ORG 





CARDINAL 19,044
DATE 





MONEY 


CARDINAL 10,309
CARDINAL 24,884
PERSON ﻿
OR

GPE 

GPE 

GPE 

GPE 

GPE 

ORG Frontier
GPE 

ORG Indemnitee
CARDINAL 18
ORG Headings
GPE References
GPE 

ORG Pronouns
GPE 

GPE 

GPE 

CARDINAL 10
CARDINAL 19
ORG 





GPE 

GPE 

GPE Delaware
GPE 

GPE Delaware
GPE 

EVENT this
Agreement
ORG Indemnitee
EVENT this
Agreement
CARDINAL one
GPE 

ORG Agreement
ORG Frontier
ORG Frontier
GPE 

ORG Indemnitee
GPE 

ORG Indemnitee
GPE 

ORG Indemnitee
GPE 

ORG Indemnitee
ORG Expenses
GPE 

ORG Frontier
GPE 

GPE Indemnitee
GPE 

ORG Frontier
GPE 

GPE 

GPE 

ORG Expenses
GPE 

CARDINAL two
GPE 

GPE 

GPE 

GPE 

ORG Frontier
GPE Indemnitee
ORGORG 


Illinois




Frontier Communications Services Inc.
GPE Arizona
ORG Frontier Directory Services Company
ORG LLC


Delaware





GPE Florida
ORG LLC


Florida





ORG Frontier Infoservices Inc.
GPE 



GPE Delaware
ORG Frontier Midstates Inc.
ORG 


Georgia
ORG LLC


Delaware





GPE Wisconsin
ORG Frontier Security Company
ORG 


Delaware





ORG Frontier Services Corp.
ORG 


Connecticu

In [ ]:
df = pd.read_html(str(table), header=0)[0]

In [141]:
text = soup.strip(' \t\n\r')
text = text.strip()

In [142]:
text

'UNITED STATES SECURITIES AND EXCHANGE\nCOMMISSION\nWashington, D.C. 20549\nFORM 10-K\n\ufeff\n☒ ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE\nSECURITIES EXCHANGE ACT OF 1934\nFor the fiscal year ended December 31, 2017\nor\n\ufeff\n☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF\nTHE SECURITIES EXCHANGE ACT OF 1934\nFor the transition period from _________ to ___________\n\ufeff\nCommission file number 001-11001\n\ufeff\n\n\ufeff\nFRONTIER COMMUNICATIONS CORPORATION\n(Exact name of registrant as specified in its charter)\n\ufeff\n\n\n\n\n\ufeff\n\n\n\n\n\n\n\n\n\n\nDelaware\n\n\n\n\n\n06-0619596\n\n\n\n\n(State or other jurisdiction of\n\n\n\n\n\n(I.R.S. Employer Identification No.)\n\n\n\n\nincorporation or organization)\n\n\n\n\n\n\n\n\n\n\n\ufeff\n\n\n\n\n\n\n\n\n\n\n401 Merritt 7\n\n\n\n\n\n\n\n\n\n\nNorwalk, Connecticut\n\n\n\n\n\n06851\n\n\n\n\n(Address of principal executive offices)\n\n\n\n\n\n(Zip Code)\n\n\n\n\n\ufeff\n\n\n\n\n\n\n\n\n\n\nRegistrant\'s telepho

In [1]:
'''doc = nlp(soup)

# Print all of the found entities and their labels
for ent in doc.ents:
    print(ent.label_, ent.text)
'''

'doc = nlp(soup)\n\n# Print all of the found entities and their labels\nfor ent in doc.ents:\n    print(ent.label_, ent.text)\n'

In [2]:
import html2text

path = "data/nc_validation_filings_test/"
file = "0000007332-18-000016.html"
html = open(path+file, encoding="utf8").read()
#print html2text.html2text(html)